# Zastosowanie algorytmu NSGA2 dla Problemu Plecakowego

**Dane**
1. Zbiór przedmiotów - każdy posiada wagę oraz wartość z przedziału \[0, 1).
2. Plecak posiadają znaną, skończoną pojemność k.
3. Wybierany jest podzbiór danego zbioru przedmiotów, mieszczący się w plecaku - suma wag mniejsza, bądź równa k.

**Cel**

Maksymalizacja sumarycznej wartości wybranych przedmiotów.

In [324]:
from random import random, randrange

**Algorytm NSGA2**

Dane: zbiór przedmiotów, ilość generacji, rozmiar populacji, k oraz prawdopodobieństwo mutacji.

1. Generowana jest populacja początkowa o danym rozmiarze (_Chromosome.random_) w postaci chromosomów reprezentowanych przez listy bitów, których długość jest równa ilości przedmiotów.
2. Dla każdej kolejnej generacji wykonywane są w pętli kolejne punkty.
3. Wykonywana jest operacja selekcji turniejowej - powstaje lista par rodziców.
4. Wykonywana jest operacja crossoveru - generowany jest zbiór dzieci.
5. Wykonywana jest operacja mutacji dzieci - z zadanym prawdopodobieństwem. Dzieci dodawane są do bieżącej populacji.
6. Wykonywany jest algorytm "Non-dominated sorting".
7. Wykonywany jest algorytm "Crowding distance sorting".
8. Usuwane są duplikaty z populacji (celem optymalizacji).

Otrzymany wynik to posortowana lista chromosomów, zatem pierwszy z nich jest "najlepszym" osobnikiem.

### Klasy modelowe

In [325]:
class Item:
    def __init__(self, weight, value):
        self.weight = weight
        self.value = value
    
    def __str__(self):
        return f"Weight: {self.weight} | Value: {self.value}"

In [326]:
class Chromosome:
    def __init__(self, size, selected=[]):
        self.size = size
        self.selected = selected

    @staticmethod
    def random(size):
        selected = []
        for i in range(size):
            selected.append(random() >= 0.5)
        return Chromosome(size, selected)

    def weightFitness(self, items):
        sum = 0
        for item in self.getSelectedItems(items):
            sum += item.weight
        return sum

    def valueFitness(self, items, maxWeight):
        sum = 0
        for item in self.getSelectedItems(items):
            sum += item.value
        return -1 if self.weightFitness(items) > maxWeight else sum

    def getSelectedItems(self, items):
        selectedItems = []
        for el in zip(self.selected, items):
            if el[0]:
                selectedItems.append(el[1])
        return selectedItems

    def dominates(self, other, items, maxWeight):
        thisValue = self.valueFitness(items, maxWeight)
        otherValue = other.valueFitness(items, maxWeight)
        if thisValue > otherValue:
            return 1
        elif thisValue < otherValue:
            return -1
        else:
            return 0

    def clone(self):
        return Chromosome(self.size, self.selected.copy())

    def __str__(self):
        return f"{self.size} {self.selected}"

    def isEqual(self, obj):
        return self.size == obj.size and self.selected == obj.selected

    @staticmethod
    def removeDuplicated(chromosomes):
        newList = []

        def exists(x):
            for el in newList:
                if el.isEqual(x):
                    return True
            return False

        for c in chromosomes:
            if not exists(c):
                newList.append(c)
        return newList

In [327]:
class ChromosomeWrapper:
    def __init__(self, chromosome):
        self.chromosome = chromosome

    @staticmethod
    def fromChromosomes(chromosomes):
        pass

    @staticmethod
    def toChromosomes(wrapperList):
        chromosomes = []
        for wrapper in wrapperList:
            chromosomes.append(wrapper.chromosome)
        return chromosomes

### Operacje genetyczne

1. **Crossover** - wybierany jest punkt podziału, geny przed tym podziałem są dziedziczone po rodzicu 1, a za podziałem po rodzicu 2.
2. **Mutation** - z danym prawdopodobieństwem odwracane są poszczególne geny.
3. **TournamentSelection** - wybieramy dwa losowe chromosomy i wybieramy dominujący. Remis jest rozpatrywany jako rzut monetą.

In [328]:
class GeneticOperations:

    def __init__(self, mutationProbability, items):
        self.mutationProbability = mutationProbability
        self.items = items

    def crossover(self, parent1, parent2):
        selected = []
        sliceIndex = randrange(parent1.size)
        for i in range(sliceIndex):
            selected.append(parent1.selected[i])
        for i in range(sliceIndex, parent1.size):
            selected.append(parent1.selected[i])
        return Chromosome(parent1.size, selected)

    def mutation(self, chromosome):
        selected = []
        for i in range(chromosome.size):
            if random() >= self.mutationProbability:
                selected.append(not chromosome.selected[i])
            else:
                selected.append(chromosome.selected[i])
        return Chromosome(chromosome.size, selected)

    def tournamentSelection(self, chromosomes, maxWeight):
        idx1 = randrange(len(chromosomes))
        idx2 = randrange(len(chromosomes))
        dominate = chromosomes[idx1].dominates(chromosomes[idx2], self.items, maxWeight)
        if dominate == 1:
            return chromosomes[idx1]
        elif dominate == -1:
            return chromosomes[idx2]
        elif random() < 0.5:
            return chromosomes[idx1]
        else:
            return chromosomes[idx2]

    def selection(self, chromosomes, maxWeight):
        result = []
        for i in range(len(chromosomes)):
            result.append([self.tournamentSelection(chromosomes, maxWeight), self.tournamentSelection(chromosomes, maxWeight)])
        return result

### Crowding distance sorting

1. Wszystkie elementy są inicjalizowane z crowdingDistance = 0.
2. Elementy są sortowane po fitnessie. Następnie ustawiane jest dla max crowdingDistance = inf.
3. Elementy ze środka przedziału \[min, max\] mają ustawiane crowdingDistance zgodnie z metryką taksówkową względem wartości fitnessu sąsiadów.
4. Elementy są sortowane malejąco względem crowdingDistance - NSGA odrzuci pewną prawą część listy.

In [329]:
class CrowdingDistanceChromosomeWrapper(ChromosomeWrapper):
    def __init__(self, chromosome):
        super().__init__(chromosome)
        self.crowdingDistance = 0

    @staticmethod
    def fromChromosomes(chromosomes):
        cdList = []
        for c in chromosomes:
            cdList.append(CrowdingDistanceChromosomeWrapper(c))
        return cdList

In [330]:
class CrowdingDistanceSort:
    def __init__(self, items):
        self.items = items

    def sort(self, cdList, maxWeight):
        cdList.sort(key=lambda cd: cd.chromosome.valueFitness(self.items, maxWeight))
        cdList[-1].crowdingDistance = float('inf') 
        for i in range(1, len(cdList) - 1):
                cdList[i].crowdingDistance = cdList[i + 1].chromosome.valueFitness(self.items, maxWeight)
                cdList[i].crowdingDistance -= cdList[i - 1].chromosome.valueFitness(self.items, maxWeight)

        cdList.sort(reverse=True, key=lambda cd: cd.crowdingDistance)
        return cdList

### Non-dominatinated sorting

1. Rozpatrujemy dla każdej pary elementów z populacji, który chromosom nad którym dominuje (uaktualniamy listę chromosomów _dominatedBy_, przez które bieżący jest zdominowany oraz liczbę _dominationCount_ wskazującą nad iloma bieżący dominuje).
2. Wybieramy z populacji wszystkie niedominujące elementy. Odpowiednio zmniejszamy _dominationCount_ dla elementów, które nad nimi dominowały.
3. Do wynikowej listy dodajemy taki podzbiór populacji, a następnie powtarzamy kroki 2-3 do momentu, gdy w populacji nie będzie istniał żaden element niedominujący (_dominationCount=0_) lub wszystkie elementy populacji zostaną rozpatrzone.

In [331]:
class NonDominatedSortChromosomeWrapper(ChromosomeWrapper):
    def __init__(self, chromosome):
        super(NonDominatedSortChromosomeWrapper, self).__init__(chromosome)
        self.dominationCount = 0
        self.dominatedBy = []

    @staticmethod
    def fromChromosomes(chromosomes):
        ndsList = []
        for c in chromosomes:
            ndsList.append(NonDominatedSortChromosomeWrapper(c))
        return ndsList

In [332]:
class NonDominatedSort:
    def __init__(self, items):
        self.items = items

    def sort(self, ndsList, maxWeight):
        frontList = []
        for i in range(len(ndsList) - 1):
            for j in range(i + 1, len(ndsList)):
                dominateResult = ndsList[i].chromosome.dominates(ndsList[j].chromosome, self.items, maxWeight)
                if dominateResult == 1:
                    ndsList[j].dominationCount += 1
                    ndsList[i].dominatedBy.append(ndsList[j])
                elif dominateResult == -1:
                    ndsList[i].dominationCount += 1
                    ndsList[j].dominatedBy.append(ndsList[i])
        nonDominantExists = True
        while nonDominantExists:
            nonDominantExists = False
            currentFront = []           
            dominatedList = []
            for i in range(len(ndsList)):
                if ndsList[i].dominationCount == 0:
                    nonDominantExists = True
                    ndsList[i].dominationCount = -1
                    currentFront.append(ndsList[i])
                    for dominated in ndsList[i].dominatedBy:
                        dominatedList.append(dominated)
            for dominated in dominatedList:
                dominated.dominationCount -= 1
            if len(currentFront) > 0:
                frontList.append(currentFront)
        return frontList

### NSGA2

In [333]:
class Nsga:
    def __init__(self, mutationProbability, items):
        self.items = items
        self.geneticOperations = GeneticOperations(mutationProbability, items)
        self.CrowdingDistanceSort = CrowdingDistanceSort(items)
        self.nonDominatedSort = NonDominatedSort(items)

    def run(self, nrGeneration, populationSize, maxWeight):
        chromosomes = []
        for i in range(populationSize):
            chromosomes.append(Chromosome.random(len(self.items)))
        for currentGeneration in range(nrGeneration):
            for i in range(len(chromosomes), populationSize):
                chromosomes.append(Chromosome.random(len(self.items)))
            selectedPairParents = self.geneticOperations.selection(chromosomes, maxWeight)
            children = []
            for selectedParents in selectedPairParents:
                children.append(self.geneticOperations.crossover(selectedParents[0], selectedParents[1]))

            mutated = list(map(lambda child: self.geneticOperations.mutation(child), children))
            children = Chromosome.removeDuplicated(mutated)

            chromosomes = [*chromosomes, *children]
            fronts = self.nonDominatedSort.sort(NonDominatedSortChromosomeWrapper.fromChromosomes(chromosomes), maxWeight)
            newPopulation = []
            currentFront = 0
            while currentFront < len(fronts) and len(newPopulation) + len(fronts[currentFront]) <= populationSize:
                newPopulation = [*newPopulation, *NonDominatedSortChromosomeWrapper.toChromosomes(fronts[currentFront])]
                currentFront += 1
            selected = self.CrowdingDistanceSort.sort(
                CrowdingDistanceChromosomeWrapper.fromChromosomes(NonDominatedSortChromosomeWrapper.toChromosomes(fronts[currentFront])), maxWeight)
            newPopulation = [*newPopulation, *CrowdingDistanceChromosomeWrapper.toChromosomes(
                selected[:(populationSize - len(newPopulation))])]
            chromosomes = Chromosome.removeDuplicated(newPopulation)[:min(populationSize, len(newPopulation))]
        return chromosomes

### Test

In [334]:
def test(itemsNumber, mutationProbability, generationsNumber, populationSize, maxWeight):
    items = []
    for i in range(itemsNumber):
        item = Item(random(), random())
        items.append(item)
        print(f"{i+1}. {item}")

    nsga = Nsga(mutationProbability, items)
    result = nsga.run(generationsNumber, populationSize, maxWeight)
    print(result[0].valueFitness(items, maxWeight))
    print(result[0].weightFitness(items))
    print(result[0])

In [335]:
test(itemsNumber=16, mutationProbability=0.2, generationsNumber=30, populationSize=200, maxWeight=3)

1. Weight: 0.6692230952971253 | Value: 0.9267820814004168
2. Weight: 0.9574122035806898 | Value: 0.7482827408063539
3. Weight: 0.7062741979200038 | Value: 0.800371873869472
4. Weight: 0.20080064070246317 | Value: 0.7535579373595784
5. Weight: 0.5093633522226934 | Value: 0.25129507366642334
6. Weight: 0.20704732748768873 | Value: 0.5413927646800791
7. Weight: 0.7275826588389992 | Value: 0.890814341987083
8. Weight: 0.6688530615337047 | Value: 0.037962615055837645
9. Weight: 0.01329832657086949 | Value: 0.7736812040156267
10. Weight: 0.4022217265315654 | Value: 0.3408592485209635
11. Weight: 0.9050286945891522 | Value: 0.8996390011647974
12. Weight: 0.37317518965232477 | Value: 0.5294661556473864
13. Weight: 0.1060684037237285 | Value: 0.8621012430606158
14. Weight: 0.07263915080484362 | Value: 0.6777605245020927
15. Weight: 0.7611578483400934 | Value: 0.6794669723370649
16. Weight: 0.01223830549109417 | Value: 0.9168943360841489
6.6447739923422935
2.813053096905041
16 [False, False, Fal